<a href="https://colab.research.google.com/github/fladdict/stable-diffusion/blob/main/super_simple_stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SSSD, Super Simple Stable Diffusion

Lightwaight easy to use Stable Diffusion Kit.
Just run all cells. Only you need is [Hugging Face's user setting page](https://huggingface.co/settings/tokens).

## Setup Google Drive (If needed)

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Install Library

In [1]:
#Install Lib
!pip install diffusers==0.4.0 transformers scipy ftfy

## Login to Hugging Face

Ges ACCESS TOKEN from [Hugging Face's user setting page](https://huggingface.co/settings/tokens).

Run a cell below and fill ACCESS TOKEN into the form.

In [2]:
#Get access token from Hugging Face
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


## Function Setup

In [9]:
import torch
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
import numpy as np
import PIL
import gc
import os

def save_image(directory_path, file_name, img):
  os.makedirs(directory_path, exist_ok = True)
  img.save(f"{directory_path}/{file_name}")



def clear_memory():
  gc.collect()
  torch.cuda.empty_cache()



def get_txt2img_pipeline(model_path = "CompVis/stable-diffusion-v1-4"):
  pipeline = StableDiffusionPipeline.from_pretrained(model_path, 
                                                     revision="fp16", 
                                                     torch_dtype=torch.float16,
                                                     use_auth_token=True)
  pipeline.to("cuda")
  pipeline.enable_attention_slicing()
  return pipeline




def get_img2img_pipeline(model_path = "CompVis/stable-diffusion-v1-4"):
  pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(model_path, 
                                                            revision="fp16", 
                                                            torch_dtype=torch.float16,
                                                            use_auth_token=True)
  pipeline.to("cuda")
  pipeline.enable_attention_slicing()
  return pipeline

pipe = None
last_model_path = None

# Diffuse!!!

Once you setup, just run only this cell

In [ ]:
from torch import autocast
from datetime import datetime
import random

#@markdown ## General Settings
prompt = "reimu hakurei, concept art, high quality, beautiful, pixiv trending" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
width = 512 #@param {type:"integer"}
height = 512 #@param {type:"integer"}

guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
seed = None #@param
num = 3 #@param {type:"integer"}

#@markdown -
#@markdown ## Init Image settings
#@markdown File path used for base Image. 
init_image = "" #@param {type:"string"}
strength = 0.8 #@param {type:"number"}

#@markdown -
#@markdown ## Model Settings
#@markdown Select your model or input your model path

model_path = "CompVis/stable-diffusion-v1-4" #@param {type:"string"} ["CompVis/stable-diffusion-v1-4", "rinna/japanese-stable-diffusion", "hakurei/waifu-diffusion", "hakurei/waifu-diffusion-v1-3", "doohickey/trinart-waifu-diffusion-50-50", "sd-dreambooth-library/disco-diffusion-style"] {allow-input: true}

#@markdown -
#@markdown ## Export Settings

save_directory_path = "/content/output/test" #@param {type:"string"}
save_file_prefix = "SD" #@param {type:"string"}
is_save_image = True #@param {type:"boolean"}
is_display_image = True #@param {type:"boolean"}

clear_memory()

with autocast("cuda"):
  for index in range(num):
    _seed = random.randint(0, 2**32) if seed == None else seed
    generator = torch.Generator("cuda").manual_seed(_seed)

    if init_image != None and init_image != "" :
      if pipe == None or type(pipe) != StableDiffusionImg2ImgPipeline or model_path != last_model_path:
        pipe = get_img2img_pipeline(model_path)
        last_model_path = model_path
      image = pipe(
          init_image = init_image,
          prompt = prompt,
          negative_prompt = negative_prompt,
          width = width,
          height = height,
          strength = strength,
          guidance_scale = guidance_scale,
          num_inference_steps = num_inference_steps,
          generator = generator
      )["sample"][0]
    else:
      if pipe == None or type(pipe) != StableDiffusionPipeline or model_path != last_model_path:
        pipe = get_txt2img_pipeline(model_path)
        last_model_path = model_path
      image = pipe(
          prompt = prompt,
          negative_prompt = negative_prompt,
          width = width,
          height = height,
          strength = strength,
          guidance_scale = guidance_scale,
          num_inference_steps = num_inference_steps,
          generator = generator
      )["sample"][0]

    if is_display_image == True:
      display(image)

    if is_save_image == True:
      file_id = datetime.today().strftime('%Y-%m-%d-%H-%M-%S')
      save_image(save_directory_path, f"{save_file_prefix}-{file_id}.png", image)



# Cheat Sheet



## Prompt for beginners


```
<MEDIUM> of <THEME> by <ARTIST> in the style of <STYLE>, <MODIFIER>, <MODIFIER>
```

```
Oil painting of landscape by Rembrandt in the style of impressionism, high quality, masterpiece
```

## Style
* Ancient Egyptian Art
* Ancient Greek Art
* Byzantine Art
* Gothic
* Medieval Art
* Renaissance
* Mannerism
* High Renaissance
* Baroque
* Rococo
* Classicism
* Neo Classicism
* Academicism
* Romanticism
* Orientalism
* Realism
* Primitivism
* Surrealism
* Impressionism
* Symbolism
* Pointillism
* Abstract Art
* Japonism
* Art Deco
* Fauvism
* Art Nouveau
* Expressionism
* Cubism
* Futurism
* Dada
* Constructivism
* Outsider art
* Concretism
* Neo Expressionism
* Pop Art
* Modernism
* Minimalism
* Photorealism
* Hyper-Realism

## Artist
* Rembrandt


## Modifier
* Masterpiece of
* High quality
* Photorealistic
* Unrealengine
* Octane Render
* 8k
* Insane detailing
* Epic